# task2

In [1]:
import json
import numpy as np
import nltk
import random
from sklearn.metrics import mean_absolute_error, mean_squared_error
from collections import defaultdict, OrderedDict
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeRegressor
from nltk.corpus import stopwords
from string import punctuation


import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def readJson(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return df

df = readJson('assignment1/train.json.gz')

In [2]:
totalData = [df[x] for x in range(0,200000)]

In [3]:
# np.random.seed(1)
# np.random.shuffle(df)
trainingData = [df[x] for x in range(0,200000)]
validationData = [df[x] for x in range(100000,200000)]

In [4]:
userIndex = {}
users = []
userToItem = defaultdict(list)
itemIndex = {}
items = []
itemToUser = defaultdict(list)
i,j = 0,0
for data in trainingData:
    userToItem[data['reviewerID']].append(data['itemID'])
    itemToUser[data['itemID']].append(data['reviewerID'])
    if data['reviewerID'] not in userIndex:
        userIndex[data['reviewerID']] = i
        users.append(data['reviewerID'])
        i += 1
    if data['itemID'] not in itemIndex:
        itemIndex[data['itemID']] = j
        items.append(data['itemID'])
        j += 1
userItemNum = defaultdict(int)
for user in userToItem:
    userItemNum[user] = len(userToItem[user])
Rui = np.zeros((len(users), len(items)))
for data in trainingData:
    Rui[userIndex[data['reviewerID']]][itemIndex[data['itemID']]] = data['rating']

In [5]:
import operator
user_sorted = list(sorted(userItemNum, key = lambda x : userItemNum[x], reverse = False))

In [48]:
# validationPredictions = []
# validationRatings = []
# for data in validationData:
#     u,i = data['reviewerID'], data['itemID']
#     prediction = []
#     if u in userAverageRating:
#         prediction.append(userAverageRating[u])      
#     if i in itemAverageRating:
#         prediction.append(itemAverageRating[i])
#     elif u not in userAverageRating:
#         prediction.append(globalAverageRating)
#     validationPredictions.append(np.mean(prediction))
#     validationRatings.append(data['rating'])
# mean_squared_error(validationPredictions, validationRatings)

In [47]:
def gradientDescent(weight_u, weight_1, regularization =1):
    alpha = random.random()
    np.random.seed(0)
    beta_u = np.random.rand(len(userIndex.keys()))
    beta_i = np.random.rand(len(itemIndex.keys()))
#     beta_u = np.ones(len(userIndex.keys()))
#     beta_i = np.ones(len(itemIndex.keys()))
    maxIter = 1e5
    numIter = 0
    alpha_path = []
    beta_u_path = []
    beta_i_path = []
    trainingMSE_path = []
    validationMSE_path = []
    while numIter < maxIter:
        alpha = np.sum([data['rating'] - weight_u*beta_u[userIndex[data['reviewerID']]] - \
                        weight_i*beta_i[itemIndex[data['itemID']]] for data in trainingData]) / len(trainingData)
        beta_u = np.array([np.sum([Rui[userIndex[user]][itemIndex[item]] - alpha - \
                                   weight_i*beta_i[itemIndex[item]] for item in userToItem[user]])
                  /(regularization + len(userToItem[user])) for user in users])
        beta_i = np.array([np.sum([Rui[userIndex[user]][itemIndex[item]] - alpha - \
                                   weight_u*beta_u[userIndex[user]] for user in itemToUser[item]])
                  /(regularization + len(itemToUser[item])) for item in items])

        trainingMSE = mean_squared_error([alpha + beta_u[userIndex[data['reviewerID']]] + \
                        beta_i[itemIndex[data['itemID']]] for data in trainingData], \
                                        [data['rating'] for data in trainingData])
        # Calculate validation MSE
#         prediction = np.zeros(len(validationData)) + alpha
#         i = 0
#         for data in validationData:
#             if data['reviewerID'] in userIndex:
#                 prediction[i] += beta_u[userIndex[data['reviewerID']]]
#             if data['itemID'] in itemIndex:
#                 prediction[i] += beta_i[itemIndex[data['itemID']]]
#             i += 1
#         validationMSE = mean_squared_error(prediction, [data['rating'] for data in validationData])
        validationMSE = trainingMSE + regularization*(np.sum(beta_u**2) + np.sum(beta_i**2))/len(trainingData)
        
        numIter += 1 
        trainingMSE_path.append(trainingMSE)
        validationMSE_path.append(validationMSE)
        alpha_path.append(alpha)
        beta_u_path.append(beta_u)
        beta_i_path.append(beta_i)
        if numIter % 20 == 0:
            print trainingMSE, validationMSE
        if len(validationMSE_path) >= 6 and validationMSE_path[-5] <= validationMSE_path[-4] <= \
            validationMSE_path[-3] <= validationMSE_path[-2] <= validationMSE_path[-1]:
            print 'Iteration:', numIter, 'Training MSE =', trainingMSE_path[-5], \
                  ', Validation MSE =', validationMSE_path[-5]
                
            return alpha_path[-5], beta_u_path[-5], beta_i_path[-5], trainingMSE_path[-5], validationMSE_path[-5]

    return alpha, beta_u, beta_i, trainingMSE, validationMSE

In [49]:
alpha, beta_u, beta_i, trainingMSE, validationMSE = gradientDescent( 1, 1,regularization = 5.6)

Iteration: 17 Training MSE = 0.808150448501 , Validation MSE = 0.932087474461


In [50]:
predictions = open("assignment1/predictions_Rating.txt", 'w')
for l in open("assignment1/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    this_beta_u, this_beta_i = 0,0

                                         
    if u in userIndex:
        this_beta_u += 1.13*beta_u[userIndex[u]]
    elif i in itemIndex:
        this_beta_i += 1.13*beta_i[itemIndex[i]]
    if i in itemIndex:
        this_beta_i += 0.87*beta_i[itemIndex[i]]
    elif u in userIndex:
        this_beta_u += 0.87*beta_u[userIndex[u]]
    p = alpha + this_beta_u + this_beta_i
    if p>5:
        p = 5
    if p<1:
        p = 1
    predictions.write(u + '-' + i + ',' + str(p) + '\n')

predictions.close()

In [ ]:
predictions = open("assignment1/predictions_Rating.txt", 'w')
for l in open("assignment1/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    this_beta_u, this_beta_i = 0,0
    if u in userLevel1:
        this_beta_u = beta_u[userIndex[u]]
    if i in userLevel2:
        this_beta_i = beta_i[itemIndex[i]]
    p = alpha + this_beta_u + this_beta_i
    if p>5.0:
        print 'da',
    predictions.write(u + '-' + i + ',' + str(min(p,5)) + '\n')

predictions.close()

In [ ]:
def gradientDescent2(regularization = 1, iniStepSize = 0.01, T = 10, k = 3):
    alpha = random.random()
    np.random.seed(0)
    
    beta_u = np.random.rand(len(userIndex.keys()))
    beta_i = np.random.rand(len(itemIndex.keys()))
    r_u = np.random.rand(len(userIndex.keys()))
    r_i = np.random.rand(len(itemIndex.keys()))
#     beta_u = np.ones(len(userIndex.keys()))
#     beta_i = np.ones(len(itemIndex.keys()))
    maxIter = 1e3
    numIter = 0
    alpha_path = []
    beta_u_path = []
    beta_i_path = []
    r_u_path = []
    r_i_path = []
    trainingMSE_path = []
    validationMSE_path = []
    while numIter < maxIter:
        print numIter
        stepSize = iniStepSize /(1+numIter*1./T)
        
        alpha = alpha - stepSize*(2*np.sum([alpha + beta_u[userIndex[data['reviewerID']]] + \
                                            beta_i[itemIndex[data['itemID']]] + \
                                            np.dot(r_u[userIndex[data['reviewerID']]], r_i[itemIndex[data['itemID']]].T)\
                                            - data['rating'] \
                                           for data in trainingData]) / len(trainingData) + 2*regularization*alpha)
        
        beta_u = beta_u - stepSize*(2*np.array([np.mean([alpha + beta_u[userIndex[user]] + beta_i[itemIndex[item]] + \
                                                        np.dot(r_u[userIndex[user]], r_i[itemIndex[item]].T) - \
                                                        Rui[userIndex[user]][itemIndex[item]] for item in userToItem[user]])\
                                                for user in users])  + 2*regularization*beta_u)

        beta_i = beta_i - stepSize*(2*np.array([np.mean([alpha + beta_u[userIndex[user]] + beta_i[itemIndex[item]] + \
                                                        np.dot(r_u[userIndex[user]], r_i[itemIndex[item]].T) - \
                                                        Rui[userIndex[user]][itemIndex[item]] for user in itemToUser[item]]) \
                                                for item in items])  + 2*regularization*beta_i)
        
        r_u = r_u - stepSize*(np.array([np.mean([2*r_i[itemIndex[item]]*\
                                                (alpha+beta_u[userIndex[user]]+beta_i[itemIndex[item]]+\
                                                 np.dot(r_u[userIndex[user]], r_i[itemIndex[item]].T)-\
                                                 Rui[userIndex[user]][itemIndex[item]]) \
                                                for item in userToItem[user]], axis = 0) \
                                        for user in users])\
                              +2*regularization*r_u)
        
        r_i = r_i - stepSize*(np.array([np.mean([2*r_u[userIndex[user]]*\
                                                (alpha+beta_u[userIndex[user]]+beta_i[itemIndex[item]]+\
                                                 np.dot(r_u[userIndex[user]], r_i[itemIndex[item]].T)-\
                                                 Rui[userIndex[user]][itemIndex[item]]) \
                                                for user in itemToUser[item]], axis = 0) \
                                        for item in items])\
                              +2*regularization*r_i)
        
        trainingMSE = mean_squared_error([alpha + beta_u[userIndex[data['reviewerID']]] + \
                        beta_i[itemIndex[data['itemID']]] + \
                               np.dot(r_u[userIndex[data['reviewerID']]], r_i[itemIndex[data['itemID']]].T) for data in trainingData], \
                                        [data['rating'] for data in trainingData])
        # Calculate validation MSE
        prediction = np.zeros(len(validationData)) + alpha
        i = 0
        for data in validationData:
            if data['reviewerID'] in userIndex:
                prediction[i] += beta_u[userIndex[data['reviewerID']]]
            else:
                prediction[i] += np.mean(beta_u)
            if data['itemID'] in itemIndex:
                prediction[i] += beta_i[itemIndex[data['itemID']]]
            else:
                prediction[i] += np.mean(beta_i)
            if data['reviewerID'] in userIndex and data['itemID'] in itemIndex:
                prediction[i] += np.dot(r_u[userIndex[data['reviewerID']]], r_i[itemIndex[data['itemID']]].T)
            i += 1
        validationMSE = mean_squared_error(prediction, [data['rating'] for data in validationData])
        print trainingMSE,validationMSE
        numIter += 1 
        trainingMSE_path.append(trainingMSE)
        validationMSE_path.append(validationMSE)
        alpha_path.append(alpha)
        beta_u_path.append(beta_u)
        beta_i_path.append(beta_i)
        r_u_path.append(r_u)
        r_i_path.append(r_i)
        if len(validationMSE_path) >= 6 and validationMSE_path[-5] <= validationMSE_path[-4] <= \
            validationMSE_path[-3] <= validationMSE_path[-2] <= validationMSE_path[-1]:
            print 'Iteration:', numIter, 'Training MSE =', trainingMSE_path[-5], \
                  ', Validation MSE =', validationMSE_path[-5]
                
            return alpha_path[-5], beta_u_path[-5], beta_i_path[-5], r_u_path[-5], r_i_path[-5], trainingMSE_path[-5], validationMSE_path[-5]

    return alpha, beta_u, beta_i, trainingMSE, validationMSE

In [ ]:
alpha, beta_u, beta_i, r_u, r_i, trainingMSE, validationMSE = \
gradientDescent2(regularization=1, iniStepSize=0.5, T=100)